In [24]:
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder.master("local").getOrCreate()

In [31]:
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'),
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

spark 내부적으로 타입을 정의할 것

In [32]:
stock_schema = ['name', 'ticker', 'country', 'price', 'currency']
df = spark.createDataFrame(data=stocks, schema=stock_schema)

데이터 구조 확인

In [33]:
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [34]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [35]:
df.createOrReplaceTempView("stocks")

In [36]:
spark.sql("select name from stocks").show()

+-------+
|   name|
+-------+
| Google|
|Netflix|
| Amazon|
|  Tesla|
|Tencent|
| Toyota|
|Samsung|
|  Kakao|
+-------+



In [37]:
spark.sql("select name, price from stocks").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
|Netflix|   645|
| Amazon|  3518|
|  Tesla|  1222|
|Tencent|   483|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [38]:
spark.sql("select name, price from stocks where country = 'Korea'").show()

+-------+------+
|   name| price|
+-------+------+
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [39]:
spark.sql("select name, price from stocks where price > 2000 ").show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
| Amazon|  3518|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [40]:
spark.sql("select name, price from stocks where price > 2000 and country == 'USA'").show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



LIKE

In [41]:
spark.sql("select name, price from stocks where country LIKE 'U%'").show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [42]:
spark.sql("select name, price from stocks where country LIKE 'U%' and name NOT LIKE '%e%'").show()

+------+-----+
|  name|price|
+------+-----+
|Amazon| 3518|
+------+-----+



In [43]:
# 1000 ~ 10000
spark.sql("select name, price from stocks where price between 1000 and 10000").show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
| Tesla| 1222|
|Toyota| 2006|
+------+-----+



In [44]:
spark.sql("select name, price from stocks where country LIKE 'USA'").show()

+-------+-----+
|   name|price|
+-------+-----+
| Google| 2984|
|Netflix|  645|
| Amazon| 3518|
|  Tesla| 1222|
+-------+-----+



In [54]:
spark.sql("""select name, price
            from stocks
            where currency = 'USD'
            and price > (select price from stocks where name = 'Tesla')""").show()

+------+-----+
|  name|price|
+------+-----+
|Google| 2984|
|Amazon| 3518|
+------+-----+



In [55]:
spark.sql("select name, price from stocks order by price desc").show()

+-------+------+
|   name| price|
+-------+------+
|  Kakao|125000|
|Samsung| 70600|
| Amazon|  3518|
| Google|  2984|
| Toyota|  2006|
|  Tesla|  1222|
|Netflix|   645|
|Tencent|   483|
+-------+------+



이름 길이로 정렬

In [58]:
spark.sql("select name, price from stocks order by length(name)").show()

+-------+------+
|   name| price|
+-------+------+
|  Tesla|  1222|
|  Kakao|125000|
| Amazon|  3518|
| Toyota|  2006|
| Google|  2984|
|Netflix|   645|
|Samsung| 70600|
|Tencent|   483|
+-------+------+



In [60]:
spark.sql("select sum(price) from stocks where country = 'Korea'").show()

+----------+
|sum(price)|
+----------+
|    195600|
+----------+



In [61]:
spark.sql("select mean(price) from stocks where country = 'Korea'").show()

+-----------+
|mean(price)|
+-----------+
|    97800.0|
+-----------+



In [62]:
spark.sql("select count(price) from stocks where country = 'Korea'").show()

+------------+
|count(price)|
+------------+
|           2|
+------------+



In [64]:
spark.sql("select count(price) from stocks where country in ('Korea', 'USA')").show()

+------------+
|count(price)|
+------------+
|           6|
+------------+



여러 개 테이블 조인
스키마 타입 정의

`earnings`는 주가 당 수익을 의미

In [65]:
earnings = [
    ('Google', 27.99, 'USD'),
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW')
]

In [66]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

In [67]:
earnings_schema = StructType([
    StructField("name", StringType(), True),
    StructField("eps", FloatType(), True),
    StructField("currency", StringType(), True),
])

In [68]:
earnings_df = spark.createDataFrame(data=earnings, schema=earnings_schema)

In [69]:
earnings_df.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [70]:
# earnings 테이블을 spark session에 등록
earnings_df.createOrReplaceTempView("earnings")

In [71]:
# 모든 것을 추출
earnings_df.select('*').show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [72]:
# 이름이 같을 때
spark.sql("select * from stocks join earnings on stocks.name = earnings.name").show()

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [76]:
# PER / 주가 수익 비율
# price earnings ratio

# 테이블 조인 + 각 테이블의 수치 나누기 연산
spark.sql("select stocks.name, (stocks.price / earnings.eps) from stocks join earnings on stocks.name = earnings.name ").show()

+-------+------------------+
|   name|     (price / eps)|
+-------+------------------+
| Amazon| 574.8366120563447|
| Google| 106.6095042658442|
|  Kakao| 177.3049645390071|
|Netflix| 251.9531306315913|
|Samsung|39.662921348314605|
|Tencent| 43.86920889728746|
|  Tesla|  656.989242258975|
| Toyota| 8.922693419839167|
+-------+------------------+

